# Extract spectral-temporal metrics at point-locations from SHP-File
#### Input from Julian Oeser
https://github.com/baumi-berlin/ee-jupyter-contrib/blob/master/examples/getting-started/display-interactive-map.ipynb

In [1]:
import ee  # the Earth Engine Python API library
ee.Initialize()
import ipyleaflet  # an interactive mapping "widget" for the notebook
from baumiTools import baumiEE, baumiVT
import json
import ogr, osr
import os
import pandas as pd

## Load a point-shapefile, convert it into feature collection
here, we load a shapefile with the points (e.g., from training data) for which we want to extrac the spectral-temporal features from GEE.

In [2]:
def Convert_SHP_to_FC(LYR, idField, classLabelField):
	import ee
	import ogr, osr
	import json
	
	
	liste = []

	feat = LYR.GetNextFeature()
	while feat:
		geom = feat.GetGeometryRef()
	# Convert the CS to EPSG:4326
		source_SR = geom.GetSpatialReference()
		target_SR = osr.SpatialReference()
		target_SR.ImportFromEPSG(4326)
		trans = osr.CoordinateTransformation(source_SR, target_SR)
		geom.Transform(trans)
	# Get the ID
		pid = feat.GetField(idField)
		cl = feat.GetField(classLabelField)
	# Build the EE-feature via the json-conversion
		geom_json = json.loads(geom.ExportToJson())
		geom_coord = geom_json['coordinates']
		geom_EE = ee.Geometry.Point(coords=geom_coord)
	# Create feature
		eeFeat = ee.Feature(geom_EE, {"ID": pid, "Class": cl})
		liste.append(eeFeat)
	# take next feature
		feat = LYR.GetNextFeature()
	# Convert list to EE feature collection
	fc = ee.FeatureCollection(ee.List(liste))
	return fc

In [22]:
shp = baumiVT.CopyToMem("C:/Miniconda3/envs/py37/Lib/site-packages/baumiTools/geeTools/_OtherSources/_Julian/training_data/training_points_testSub3.shp")
#tempFolder = "G:/Baumann/temp/"
output = "D:/output.csv"
field = "id"
clasLabel = "wetland"
yrField = "Year"

fc = Convert_SHP_to_FC(shp.GetLayer(), field, clasLabel)

In [18]:
wetland = fc.filter(ee.Filter.eq("Class",1))
other = fc.filter(ee.Filter.eq("Class",0))

In [26]:
# test 228077
median_L8 = baumiEE.Calculate_Landsat_Seasonal_Median(year=2019, startMonth=1, endMonth=3, path=228, row=77)#, roi=geom_EE)# additional arguments: roi=feat_EE or path=227, row=28
for win in [[4,6], [7,9],[10,12]]:
    median_L8 = median_L8.addBands(baumiEE.Calculate_Landsat_Seasonal_Median(year=2019, startMonth=win[0], endMonth=win[1], path=228, row=77))
bandNames_L8 = median_L8.bandNames()

In [25]:
#trainingSample = median_L8.sampleRegions(fc,["wetland"],100)
#trainingSample
trainingSample = median_L8.sampleRegions(
  collection = fc,
  properties = [clasLabel],
  scale = 30,
  tileScale = 2
    
)
trainingSample.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'properties': {'band_order': ['B_2019-01-03',
   'G_2019-01-03',
   'R_2019-01-03',
   'NIR_2019-01-03',
   'SWIR1_2019-01-03',
   'SWIR2_2019-01-03',
   'B_2019-04-06',
   'G_2019-04-06',
   'R_2019-04-06',
   'NIR_2019-04-06',
   'SWIR1_2019-04-06',
   'SWIR2_2019-04-06',
   'B_2019-07-09',
   'G_2019-07-09',
   'R_2019-07-09',
   'NIR_2019-07-09',
   'SWIR1_2019-07-09',
   'SWIR2_2019-07-09',
   'B_2019-10-12',
   'G_2019-10-12',
   'R_2019-10-12',
   'NIR_2019-10-12',
   'SWIR1_2019-10-12',
   'SWIR2_2019-10-12']},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '0_0',
   'properties': {'B_2019-01-03': 236.5,
    'B_2019-04-06': 413.0,
    'B_2019-07-09': 784.0,
    'B_2019-10-12': 733.0,
    'G_2019-01-03': 620.0,
    'G_2019-04-06': 699.0,
    'G_2019-07-09': 1097.0,
    'G_2019-10-12': 1180.0,
    'NIR_2019-01-03': 4618.5,
    'NIR_2019-04-06': 1645.0,
    'NIR_2019-07-09': 2431.0,
    'NIR_2019-10-12': 3093.0,
    '

In [7]:
test = median_L8.reduceRegions(collection = fc,
                               reducer = ee.Reducer.mean(),
                               scale = 30,
                               tileScale = 16)
test.getInfo()

{'type': 'FeatureCollection',
 'columns': {'B_2019-01-12': 'Float<-32768.0, 32767.0>',
  'Class': 'Float',
  'G_2019-01-12': 'Float<-32768.0, 32767.0>',
  'ID': 'Long',
  'NIR_2019-01-12': 'Float<-32768.0, 32767.0>',
  'R_2019-01-12': 'Float<-32768.0, 32767.0>',
  'SWIR1_2019-01-12': 'Float<-32768.0, 32767.0>',
  'SWIR2_2019-01-12': 'Float<-32768.0, 32767.0>',
  'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-57.11458528198303, -27.121877159673907]},
   'id': '0',
   'properties': {'B_2019-01-12': 267.5,
    'Class': 0.0,
    'G_2019-01-12': 451.0,
    'ID': 444,
    'NIR_2019-01-12': 1495.0,
    'R_2019-01-12': 455.5,
    'SWIR1_2019-01-12': 1240.0,
    'SWIR2_2019-01-12': 751.0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-57.11261857660151, -27.12416058867195]},
   'id': '1',
   'properties': {'B_2019-01-12': 273.5,
    'Class': 0.0,
    'G_2019-01-12': 443.5,
    'ID': 7523,
    'NIR

In [27]:
bandNames = median_L8.bandNames()
classifier = ee.Classifier.randomForest(100,0).setOutputMode('PROBABILITY').train(trainingSample,"Class",bandNames)
classification = median_L8.classify(classifier).multiply(100)

In [28]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

Map = ipyleaflet.Map(center=(-14.438411, -63.743799), zoom=4,
                      layout={'height':'600px'},
                      scroll_wheel_zoom=True)
classific = ipyleaflet.TileLayer(name="classification", url=GetTileLayerUrl(
    classification.visualize(min=0, max=8000, gamma=1.5)))
Map.add_layer(classific)
Map.add_control(ipyleaflet.LayersControl(position='topright'))

Map

EEException: Property 'Class' of feature '0_0' is missing.